In [1]:
print('HI')

HI


In [2]:
pip install pdfplumber


  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata (48 kB)
  Using cached cryptography-43.0.1-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.6 MB 12.9 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.6 MB 9.7 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.6 MB 8.4 MB/s eta 0:00:01
   ---------------------- ----------------- 1.4/2.6 MB 8.3 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.6 MB 8.2 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.6 MB 8.0 MB/s eta 0:00:01
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pdfplumber

In [ ]:



# 2. Initialize Google Generative AI embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 3. Define function to extract and chunk PDF content
def extract_and_chunk_pdfs(pdf_files):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    all_chunks = []
    for pdf in pdf_files:
        try:
            text = ""
            with pdfplumber.open(pdf) as pdf_reader:
                for page in pdf_reader.pages:
                    text += page.extract_text()  # Extract text from each page
            chunks = text_splitter.split_text(text)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"Error processing {pdf}: {e}")
    return all_chunks

# 4. Preprocess and index PDF chunks into Pinecone, and calculate total length of embeddings
def index_pdf_content(pdf_files):
    pdf_text_chunks = extract_and_chunk_pdfs(pdf_files)
    total_embedding_length = 0  # Initialize variable to hold total length of all embeddings
    
    for i, chunk in enumerate(pdf_text_chunks):
        try:
            vector = embeddings.embed_query(chunk)
            total_embedding_length += len(vector)  # Accumulate length of each embedding vector
            index.upsert([("doc-id", vector)])  # Use a unique ID for each chunk
        except Exception as e:
            print(f"Error indexing chunk {i+1}: {e}")
    
    # Print the total combined length of embeddings for all PDF files
    print(f"Total combined length of all embeddings: {total_embedding_length}")

# 5. Preload your PDF files (ensure files are placed in a directory for processing)
pdf_files = ["document1.pdf", "document2.pdf", "document3.pdf"]  # List your PDF files here
index_pdf_content(pdf_files)
